In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import configparser
import os.path
from os import path
from importlib import reload
import wandb


creds_path_ar = ["../../credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

In [ ]:
cd {PATH_ROOT}

In [ ]:
pip install numpy==1.18.5

In [ ]:
import os
import pandas as pd
import numpy as np
from termcolor import colored
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [ ]:
from src.data import get_data, feat_data
from src.visualization import *
from src.features import add_data, augmentations, specto_feat
from src.models import *

In [ ]:
train_path = 'MAFAT RADAR Challenge - Training Set V1'
data = get_data.load_data(train_path, folder=PATH_DATA)

In [ ]:
data.keys()

In [ ]:
segment_id = 0
track_id = data['track_id'][segment_id]
iq = data['iq_sweep_burst'][segment_id]
track, _ = add_data.concatenate_track(data, track_id)
doppler = data['doppler_burst'][segment_id]
label = data['target_type'][segment_id]
angle = 10

## Homegrown Augmentations

In [ ]:
new_segments = augmentations.split_rotation(data, track_id, angle=angle, n_splits=4)

In [ ]:
plt.figure(figsize=(10, 10))
for i, augmented_image in enumerate(new_segments.values()):
    ax = plt.subplot(3, 4, i + 1)
    plt.imshow(augmented_image)
    plt.axis("off")

In [ ]:
augmented_image.shape

## Keras Augmentation

In [ ]:
image = specto_feat.calculate_spectrogram(iq)
image = tf.expand_dims(image, -1)

In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
#   layers.experimental.preprocessing.Rescaling(1./255)
])

In [ ]:
result = resize_and_rescale(image)
_ = plt.imshow(result)

In [ ]:
image = tf.expand_dims(result, 0)

In [ ]:
data_augmentation = tf.keras.Sequential([
#   layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(3):
    augmented_image = data_augmentation(image)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0])
    plt.axis("off")